In [1]:

'''

Cleaning data: Remove patients with missing sex and short EHR observation period (< 1 year)

'''

import pandas as pd
import numpy as np

# --- Load data ---
df_cohort = pd.read_csv('df_cohort.csv', dtype={
    'source_concept_code': str, 'ICDcode': str, 'Phecode': str
})

demographic_data = pd.read_csv('demographic_dataset.csv')

#### Load condition dataframe of patients from age 18-90

In [ ]:
## select patients from age 18-90; return dataset_45023455_condition_df

# This query represents dataset "whole-cohort-all-condition" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_45023455_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (
                                    137193, 137977, 3663207, 37311320, 4011630, 4021667, 4024705, 4027304, 4041287, 4042140, 40480457, 4070767, 4089228, 4094294, 4132559, 4167864, 4168498, 4172024, 4179304, 4196732, 4213737, 4274025, 4275722, 4297708, 43021226, 4303401, 432453, 433595, 437312, 437382
                                )       
                                AND full_text LIKE '%_rank1]%'      
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    age_at_consent BETWEEN 18 AND 90 
                            ) 
                        )
                )
            ) c_occurrence 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                ON c_occurrence.condition_type_concept_id = c_type.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                ON v.visit_concept_id = visit.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_45023455_condition_df = pd.read_gbq(
    dataset_45023455_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

#dataset_45023455_condition_df.head(5)

#### Load person dataframe of patients from age 18-90

In [40]:
## select patients from age 18-90; return dataset_45023455_person_df
# This query represents dataset "whole-cohort-all-condition" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_45023455_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        age_at_consent BETWEEN 18 AND 90 
                ) 
            )"""

dataset_45023455_person_df = pd.read_gbq(
    dataset_45023455_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

#dataset_45023455_person_df.head(5)


Downloading:   0%|          | 0/412573 [00:00<?, ?rows/s]

In [ ]:
# --- Compute first and last EHR record per person ---

df_cohort['condition_start_datetime'] = pd.to_datetime(
    df_cohort['condition_start_datetime'],format='mixed'
)
ehr_dates = df_cohort.groupby('person_id')['condition_start_datetime'].agg(
    first_EHR = 'min', 
    last_EHR = 'max'
)
df_cohort = df_cohort.merge(ehr_dates, on='person_id', how='left')

df_cohort['length_first_to_last'] = (
    (df_cohort['last_EHR'] - df_cohort['first_EHR']).dt.days/365
)
# --- Filter out patients with less than 1 year of data ---

person_ids_to_remove = df_cohort.loc[df_cohort['length_first_to_last'] < 1, 'person_id'].unique()
df_cohort_filter = df_cohort[~df_cohort['person_id'].isin(person_ids_to_remove)]

num_case = df_cohort_filter[df_cohort_filter['HS'] == 1]['person_id'].nunique()
num_control = df_cohort_filter[df_cohort_filter['HS'] == 0]['person_id'].nunique()

print('Number of case patients (HS=1):', num_case)
print('Number of control patients (HS=0):', num_control)

# --- Add demographic data

cohort_df= phe_condition_filter_df.merge(demographic_data,on='person_id',how='left')

# --- Calculate age at first visit ---
cohort_df['age_first_visit'] = (cohort_df['first_EHR'] - cohort_df['date_of_birth']).dt.days/365

# --- Remove patient sex missing
sub_sex = cohort_df_sub[
    (cohort_df_sub['sex_at_birth'] == 'PMI: Skip') |
    (cohort_df_sub['sex_at_birth'] == 'None')
]
remove_sex = set(sub_sex['person_id'])
cohort_df = cohort_df[~cohort_df['person_id'].isin(remove_sex)]

num_case = cohort_df[cohort_df['HS'] == 1]['person_id'].nunique()
num_control = cohort_df[cohort_df['HS'] == 0]['person_id'].nunique()

print('Number of patients in case and control after removing missing sex info:')
print('Number of case patients (HS=1):', num_case)
print('Number of control patients (HS=0):', num_control)


# --- Remove patients with age < 18 or > 90 at last EHR date ---
cohort_df['last_EHR'] = pd.to_datetime(cohort_df['last_EHR'], format = 'mixed')
cohort_df['date_of_birth'] = pd.to_datetime(cohort_df['date_of_birth'])
cohort_df['age_last_EHR'] = (cohort_df['last_EHR']- cohort_df['date_of_birth']).dt.days/365

# Get patients outside 18–90 range
age_over_90 = cohort_df[cohort_df['age_last_EHR'] > 90][['person_id']].drop_duplicates()
age_less_18 = cohort_df[cohort_df['age_last_EHR'] <18 ][['person_id']].drop_duplicates()

age_over_90 = age_over_90['person_id'].to_list()
age_less_18 = age_less_18['person_id'].to_list()

age_less_18_over_90 = age_less_18 + age_over_90

# Print counts
print('Number of patients less than 18:', len(age_less_18))
print('Number of patients over 90:', len(age_over_90))
print('Total removed due to age out of range:', len(age_less_18_over_90))

# Filter final cohort
cohort_18_to_90_df = cohort_df[~cohort_df['person_id'].isin(age_less_18_over_90)]
# Show result grouped by HS
print('\nRemaining patients by group (HS):')
print(cohort_18_to_90_df.groupby('HS')['person_id'].nunique())

cohort_18_to_90_df.to_csv('cohort_18_to_90_df.csv',index=False)

In [57]:
#cohort_df_sub.to_csv('result_v2/case_control_person_df.csv',index=False)